# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [33]:
#import logging
import os
#import csv

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

from azureml.widgets import RunDetails

from azureml.core.model import Model
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.webservice import AciWebservice, Webservice

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-145826
aml-quickstarts-145826
southcentralus
2c48c51c-bd47-40d4-abbe-fb8eabd19c8c


In [5]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'automl-heart-failure-prediction'

experiment = Experiment(ws, experiment_name)
experiment.start_logging()
experiment

Name,Workspace,Report Page,Docs Page
automl-heart-failure-prediction,quick-starts-ws-145826,Link to Azure Machine Learning studio,Link to Documentation


In [6]:
# Choose a name for your CPU cluster
amlcompute_cluster_name = "compute-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)


Creating.........
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


In [7]:
# NOTE: update the key to match the dataset name
found = False
key = "heart-failure"
description_text = "Heart Failure Prediction DataSet from UCI"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00519/heart_failure_clinical_records_dataset.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [8]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",   
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [12]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

Submitting remote run.
Running on remote compute: compute-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
automl-heart-failure-prediction,AutoML_83343dfc-b0e0-46e6-a5d2-622e085ace09,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: 

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [13]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [14]:
remote_run.wait_for_completion(show_output=True)

Experiment,Id,Type,Status,Details Page,Docs Page
automl-heart-failure-prediction,AutoML_83343dfc-b0e0-46e6-a5d2-622e085ace09,automl,Running,Link to Azure Machine Learning studio,Link to Documentation



Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missin

{'runId': 'AutoML_83343dfc-b0e0-46e6-a5d2-622e085ace09',
 'target': 'compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-05-27T09:11:43.095009Z',
 'endTimeUtc': '2021-05-27T09:33:08.514643Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'compute-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-heart-failure-prediction","subscription_id":"2c48c51c-bd47-40d4-abbe-fb8eabd19c8c","resource_group":"aml-quickstarts-145826","workspace_name":"quick-starts-ws-145826","region":"southcentralus","compute_target":"compute-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":1000

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [15]:
best_run, fitted_model = remote_run.get_output()
best_metrics = best_run.get_metrics()
print("Best Run Id : {}".format(best_run.id),
      "Accuracy : {}".format(best_metrics['accuracy']),
      "Best metrics : {}".format(best_metrics),
      "Best model : {}".format(fitted_model), sep = '\n'
     )

Best Run Id : AutoML_83343dfc-b0e0-46e6-a5d2-622e085ace09_28
Accuracy : 0.8695402298850576
Best metrics : {'recall_score_micro': 0.8695402298850576, 'f1_score_micro': 0.8695402298850576, 'balanced_accuracy': 0.8317233461618472, 'accuracy': 0.8695402298850576, 'AUC_macro': 0.9052517891126586, 'recall_score_macro': 0.8317233461618472, 'precision_score_micro': 0.8695402298850576, 'f1_score_macro': 0.8392352427243731, 'AUC_micro': 0.9159515127493725, 'weighted_accuracy': 0.8931097843494873, 'average_precision_score_weighted': 0.9213755908598529, 'f1_score_weighted': 0.8663079495790891, 'average_precision_score_micro': 0.9192011243894452, 'precision_score_weighted': 0.8785107105733925, 'average_precision_score_macro': 0.8982475877906879, 'recall_score_weighted': 0.8695402298850576, 'matthews_correlation': 0.6948525834233357, 'norm_macro_recall': 0.6634466923236946, 'AUC_weighted': 0.9052517891126588, 'precision_score_macro': 0.8650381075304349, 'log_loss': 0.38752538084708543, 'accuracy_tab

In [16]:
print(fitted_model._final_estimator)

PreFittedSoftVotingClassifier(
    estimators=[('14', Pipeline(memory=None,
             steps=[('minmaxscaler', MinMaxScaler(copy=True, feature_range=(0, 1))),
                    ('lightgbmclassifier',
                     LightGBMClassifier(boosting_type='gbdt', colsample_bytree=0.4955555555555555, learning_rate=0.06842421052631578, max_bin=20, max_depth=-1, min_child_weight=7, min_data_in_leaf=0.06896862068965517, min_split_gain=0.21052631578947367, n_estimators=4...
        iteration_timeout_param=None,
        feature_column_names=None,
        label_column_name=None,
        weight_column_name=None,
        cv_split_column_names=None,
        enable_streaming=None,
        timeseries_param_dict=None,
        gpu_training_param_dict={'processing_unit_type': 'cpu'}
    ), random_state=None, reg_alpha=0.894736842105263, reg_lambda=0.21052631578947367, subsample=0.9405263157894738))],
             verbose=False)), ('17', Pipeline(memory=None,
             steps=[('standardscalerwrap

In [17]:
#TODO: Save the best model
model = best_run.register_model(model_name = "automl-heart-failure-best-model", model_path = './outputs/model.pkl')
print(model)

Model(workspace=Workspace.create(name='quick-starts-ws-145826', subscription_id='2c48c51c-bd47-40d4-abbe-fb8eabd19c8c', resource_group='aml-quickstarts-145826'), name=automl-heart-failure-best-model, id=automl-heart-failure-best-model:1, version=1, tags={}, properties={})


In [20]:
for model in Model.list(ws):
    print(model.name, 'Version: ', model.version)

automl-heart-failure-best-model Version:  1


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [23]:
best_run.download_file('/outputs/model.pkl', os.path.join('./model','automl-heart-failure-best-model'))
for f in best_run.get_file_names():
    if f.startswith('outputs'):
        output_file_path = os.path.join('./model', f.split('/')[-1])
        print(f'Downloading from {f} to {output_file_path} ...')
        best_run.download_file(name=f, output_file_path=output_file_path)

In [24]:
model = best_run.register_model(model_name = 'automl-heart-failure-best-model', model_path = './outputs/model.pkl')

In [28]:
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'conda_env.yml')
environment = Environment.from_conda_specification(name = 'heart-failure-environment', file_path = 'conda_env.yml')

In [29]:
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')

In [32]:
inference_config = InferenceConfig(entry_script = 'score.py', environment = environment)

In [34]:
aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1, auth_enabled = True, enable_app_insights = True)

In [35]:
webservice = Model.deploy(workspace = ws, name = 'heart-failure-service', models=[model], inference_config = inference_config, deployment_config = aci_config, overwrite = True)

In [36]:
webservice

AciWebservice(workspace=Workspace.create(name='quick-starts-ws-145826', subscription_id='2c48c51c-bd47-40d4-abbe-fb8eabd19c8c', resource_group='aml-quickstarts-145826'), name=heart-failure-service, image_id=None, compute_type=None, state=ACI, scoring_uri=Transitioning, tags=None, properties={}, created_by={})

In [ ]:
webservice.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-05-27 10:05:14+00:00 Creating Container Registry if not exists.
2021-05-27 10:05:25+00:00 Registering the environment.
2021-05-27 10:05:28+00:00 Building image.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service